# Create the webapplication logic and visualisation

> This module creates the `FastHTML` webapplication and starts the server.

In [ ]:
#| default_exp webapp

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
from monsterui.all import *

from infoflow.clasdef import *
from infoflow.creinst import *
from infoflow.viz import *

## Stucture of the web-application

The web-application will show on the main page the information flow as created by the `create_combined_infoflow_viz` from the `infoflow.viz` module. This function returns a `graphviz.graphs.Digraph` object. We turn this object into a `SVG` string which we can then modify to create clickable nodes. To make this possible we need several steps:

1. Create the infoflow graph
2. Convert the graph to an SVG string
3. Create a dictionary from all the nodes in the graph
4. Use that dictionary to alter the `SVG` string to create clickable nodes
5. Create the main page from the web-application that shows the information flow as an `SVG` image.
6. Create a function to make a webpage for every element in the infoflow.

::: {callout}
Ad. 4.

We could also create this dictionary from the instances we made from all the elements in our infoflow. The same instances that are use to create the graph in the first place. But I choose to create the dictionary from the `SVG`-string from the `Digraph` object, becaust then I will be able to use that function to create clickable `SVG` images from other sources as well.
:::

#### How to visualize a `SVG` string in a webapp

Use `NotStr()` to prevent HTML escaping of the SVG string.

In [ ]:
svg_sample = """<svg width="268pt" height="338pt" viewBox="0.00 0.00 267.84 338.16" xmlns="http://www.w3.org/2000/svg">
<g id="graph0" class="graph" transform="scale(1 1) rotate(0) translate(4 334.16)">
<polygon fill="white" stroke="transparent" points="-4,4 -4,-334.16 263.84,-334.16 263.84,4 -4,4"/>
<g id="node3" class="node">
<polygon fill="lightgreen" stroke="black" points="198.09,-129.08 163.76,-160.13 95.08,-160.13 60.75,-129.08 95.08,-98.03 163.76,-98.03 198.09,-129.08"/>
<text text-anchor="middle" x="129.42" y="-132.88" font-family="Times,serif" font-size="14.00">Recall</text>
<text text-anchor="middle" x="129.42" y="-117.88" font-family="Times,serif" font-size="14.00">(retrieve)</text>
</g>
</svg>"""

In [ ]:
show(Div(NotStr(svg_sample)))

#### How to make a node from the `Digraph` `SVG`-string clickable in `FastHTML`

- Add onclick handlers with htmx.ajax calls to the <g> elements you want clickable
- Include a target div (#content-area) where content will be swapped
- Add a CSS style attribute so the pointer cursor changes to a hand when hovering over the node

For example you can add this to the <g> element of the node you want clickable:

```python
onclick="htmx.ajax('GET', '/recall-retrieve', {target: '#content-area', swap: 'outerHTML'})"
style="cursor: pointer;"
```

But a more concise way is to add CSS styling to the FastHTML app that targets all nodes:

```python
.node { cursor: pointer; }
```

That is what we will be using in this application.

Below is an example of a `SVG`-string with a clickable node that shows a changing pointer cursor when hovering over it.

In [ ]:
svg_sample_click = """<svg width="268pt" height="338pt" viewBox="0.00 0.00 267.84 338.16" xmlns="http://www.w3.org/2000/svg">
<g id="graph0" class="graph" transform="scale(1 1) rotate(0) translate(4 334.16)">
<polygon fill="white" stroke="transparent" points="-4,4 -4,-334.16 263.84,-334.16 263.84,4 -4,4"/>
<g id="node3" class="node" onclick="htmx.ajax('GET', '/recall-retrieve', {target: '#content-area', swap: 'outerHTML'})" style="cursor: pointer;">
<polygon fill="lightgreen" stroke="black" points="198.09,-129.08 163.76,-160.13 95.08,-160.13 60.75,-129.08 95.08,-98.03 163.76,-98.03 198.09,-129.08"/>
<text text-anchor="middle" x="129.42" y="-132.88" font-family="Times,serif" font-size="14.00">Recall</text>
<text text-anchor="middle" x="129.42" y="-117.88" font-family="Times,serif" font-size="14.00">(retrieve)</text>
</g>
</svg>"""

The below example can't be clicked, because it is not running on a webserver and the GET request will also fail, because the endpoint doesn't exist yet.
But it does show how the pointer changes.

In [ ]:
show(Div(NotStr(svg_sample_click), id="content-area"))

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()